# SVM vs Logistic Regression

En este Notebook se observarán las diferencias entre el SVM y la regresión logística.


<table class="tfo-notebook-buttons" align="center">

  <td>
    <a target="_blank" href="https://colab.research.google.com/github/juancop/metodos_analitica_2/blob/main/02_SVM/00_ejemplos_lineal/02_logit_vs_svm.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" /></a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/juancop/metodos_analitica_2/blob/dev/01_optimization/02_SVM/00_ejemplos_lineal/02_logit_vs_svm.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /></a>
  </td>
  
</table>

In [ ]:
# Librerías a Utilizar
import numpy as np
from sklearn import linear_model, svm
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
import imageio
import os

In [ ]:
# Simulación de Datos
np.random.seed(0)
X = np.r_[np.random.randn(200, 2) - [2, 2], np.random.randn(200, 2) + [2, 2]]
Y = [0] * 200 + [1] * 200



In [ ]:

# Datos Simulados
plt.scatter(X[:, 0], X[:, 1], c=Y, edgecolors='k', cmap=plt.cm.Paired)
plt.show()


## Parte 1: Ajuste de Modelos

In [ ]:
# Ajuste Regresión Logística
logreg = linear_model.LogisticRegression(penalty = None)
logreg.fit(X, Y)


In [ ]:

# Ajuste SVM
svc = svm.SVC(kernel='linear')
svc.fit(X, Y)


In [ ]:

# Fronteras de Decisión
plt.scatter(X[:, 0], X[:, 1], c=Y, edgecolors='k', cmap=plt.cm.Paired)

# Logit
ax = plt.gca()
xlim = ax.get_xlim()
ylim = ax.get_ylim()
xx = np.linspace(xlim[0], xlim[1], 30)
yy = np.linspace(ylim[0], ylim[1], 30)
YY, XX = np.meshgrid(yy, xx)
xy = np.vstack([XX.ravel(), YY.ravel()]).T
Z = logreg.decision_function(xy).reshape(XX.shape)
ax.contour(XX, YY, Z, colors='r', levels=[0], alpha=0.5, linestyles=['-'])

# SVM
Z = svc.decision_function(xy).reshape(XX.shape)
ax.contour(XX, YY, Z, colors='b', levels=[-1, 0, 1], alpha=0.5, linestyles=['--', '-', '--'])

# Vector de Soporte
ax.scatter(svc.support_vectors_[:, 0], svc.support_vectors_[:, 1], s=100, facecolors='none', edgecolors='k')

plt.show()


## Parte 2: Muestreo Aleatorio (sin reemplazamiento)

In [ ]:
# Si el directorio 'fit_folder' no existe, lo creamos
fit_folder = 'fit_folder'
os.makedirs(fit_folder, exist_ok=True)

# Número de muestras
num_samples = len(X) // 5

# Modelos a usar
logreg = linear_model.LogisticRegression(penalty=None)
svc = svm.SVC(kernel='linear', C = 1) # Cambiar por 0.1 para ver

for i in range(100):
    # Barajamos los datos
    X, Y = shuffle(X, Y)

    # Tomamos una muestra de los datos
    X_sample, Y_sample = X[:num_samples], Y[:num_samples]

    # Entrenamos los modelos
    logreg.fit(X_sample, Y_sample)
    svc.fit(X_sample, Y_sample)

    # Visualización de Datos
    plt.scatter(X_sample[:, 0], X_sample[:, 1], c=Y_sample, edgecolors='k', cmap=plt.cm.Paired)
    plt.xlim((-4, 4))
    plt.ylim((-4, 4))
    plt.xticks(np.arange(-4, 5, step=1)) # marcas en el eje x
    plt.yticks(np.arange(-4, 5, step=1)) # marcas en el eje y

    ax = plt.gca()
    xlim, ylim = ax.get_xlim(), ax.get_ylim()
    xx, yy = np.linspace(xlim[0], xlim[1], 30), np.linspace(ylim[0], ylim[1], 30)
    YY, XX = np.meshgrid(yy, xx)
    xy = np.vstack([XX.ravel(), YY.ravel()]).T

    # Frontera de Decisión de Regresión Logística
    Z = logreg.decision_function(xy).reshape(XX.shape)
    ax.contour(XX, YY, Z, colors='r', levels=[0], alpha=0.5, linestyles=['-'])

    # Frontera de Decisión de SVM
    Z = svc.decision_function(xy).reshape(XX.shape)
    ax.contour(XX, YY, Z, colors='b', levels=[-1, 0, 1], alpha=0.5, linestyles=['--', '-', '--'])

    # Resaltamos los vectores de soporte
    ax.scatter(svc.support_vectors_[:, 0], svc.support_vectors_[:, 1], s=100, facecolors='none', edgecolors='k')

    # Guardamos la gráfica
    plt.savefig(f'{fit_folder}/plot_{i}.png')
    plt.close()


## ¡Un GIF!

In [ ]:
# Create a list of filenames
filenames = [os.path.join('fit_folder', x) for x in os.listdir('fit_folder') if x.endswith('.png')]

# Create a list of images
images = []
for filename in filenames:
    images.append(imageio.imread(filename))


imageio.mimsave('movie.gif', images, duration=0.5)
os.system("rm fit_folder/*")